In [39]:
import requests
import json
import time
import os

API_KEY = 'ZdpeG8xFeTZRSqqiNnbe6tvcV9OI6FKenEdeuEpMLuUhq41DQ9EX9-71yKGfsIvxW9mYIzqZznCG2AuECKuQVOmGUvti485q_e_dS4OxSi3ZCSqwcABXYhdjyr9oZnYx'
HEADERS = {'Authorization': f'Bearer {API_KEY}'}
URL = 'https://api.yelp.com/v3/businesses/search'
DATABASE_FILE = 'restaurants_data.json'

def fetch_yelp_data(location, offset):
    params = {
        'term': 'restaurants',
        'location': location,
        'limit': 50,
        'offset': offset
    }
    response = requests.get(URL, headers=HEADERS, params=params)
    return response.json()

def write_to_file(data):
    with open(DATABASE_FILE, 'w') as file:
        file.write(json.dumps(data) + '\n')

location = "San Francisco, CA"
max_calls = 300
num_calls = 0
offset = 0

while num_calls < max_calls:
    data = fetch_yelp_data(location, offset)
    if not data.get('businesses'):
        print("Not valid data")
        print(data)
        continue
    write_to_file(data)
    offset += 50
    num_calls += 1
    time.sleep(0.5)

    if offset >= data['total']:
        print("No more data")
        break

print("Fnished fetching data")

Not valid data
{'error': {'code': 'INTERNAL_ERROR', 'description': 'Something went wrong internally, please try again later.'}}
Fnished fetching data


In [36]:
import pandas as pd

def parse_yelp_data(json_data):
    restaurants = []
    for business in json_data['businesses']:
        if 'price' in business:

            restaurant = {
                'name': business['name'],
                'cuisine': ', '.join([category['title'] for category in business['categories']]),
                'city' : business['location']['city'],
                'address': business['location']['address1'],
                'price_range': business.get('price', 'N/A'),
                'average_rating': business['rating'],
                'review_count': business['review_count']
            }
            restaurants.append(restaurant)
    return restaurants


parsed_data = []
with open(DATABASE_FILE, 'r') as file:
    for line in file:
        json_data = json.loads(line)
        parsed_data.extend(parse_yelp_data(json_data))

restaurants_df = pd.DataFrame(parsed_data)

print(restaurants_df.shape)

(39, 7)
